Agora o bicho pega!!!

## Primeiro Passo: **Vetorização** ##

A vetorização converte as variáveis que são analisadas pelo algoritmo em vetores, ou conjuntos unidimensionais de números.
Essa vetorização é importante para nos permitir calcular algebricamente a similaridade entre uma variável e outra

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel # Para calcular a similaridade

# --- 1. CARREGAR E PREPARAR O DF (Substitua esta linha pelo seu carregamento real) ---
df_consolidado = pd.read_csv('df_consolidado.csv') 

# Preenche valores NaN nas colunas de lista com strings vazias para evitar erros no join
df_consolidado['genres_list'] = df_consolidado['genres_list'].fillna('')
df_consolidado['keywords_list'] = df_consolidado['keywords_list'].fillna('')

# Cria uma coluna 'corpus' combinando as features de conteúdo
# Removendo o pipe '|' para que o TF-IDF trate os termos individualmente
df_consolidado['corpus'] = (df_consolidado['genres_list'].str.replace('|', ' ', regex=False) + ' ' + 
                            df_consolidado['keywords_list'].str.replace('|', ' ', regex=False))

print("Corpus de conteúdo criado. Exemplo:")
print(df_consolidado['corpus'].head(1).iloc[0])

Corpus de conteúdo criado. Exemplo:
Animation Comedy Family jealousy toy boy friendship friends rivalry boy next door new toy toy comes to life


Aqui utilizamos o algoritmo TF-IDF Vectorizer para dois objetivos:

- O algoritmo precisa aprender quais é o vocabulário, ou as palavras-chave (keywords).
- Após aprender todas as palavras-chave, o algoritmo vai aprender quais são mais raras de se repetirem e quais são mais comuns.

Ex: Gêneros como "Ação" ou palavras-chave como "Animação" vão ser mais comuns, e portanto serão entendidas como mais genéricas. O efeito oposto é causado quando palavras-chave raras como "Máquina do Tempo" ou "Traficante de Drogas", que são muito menos comuns, tornarão aquele filme mais destacável.

In [2]:
# Inicializa o TF-IDF Vectorizer
# stop_words='english' remove palavras comuns (o, a, de), o que é bom para keywords em inglês.
tfidf = TfidfVectorizer(stop_words='english')

# Constrói a matriz TF-IDF
# Cada linha é um filme, e cada coluna é uma palavra-chave/gênero (feature).
tfidf_matrix = tfidf.fit_transform(df_consolidado['corpus'])

print("\nMatriz TF-IDF construída. Formato (Filmes, Features):")
print(tfidf_matrix.shape)


Matriz TF-IDF construída. Formato (Filmes, Features):
(31030, 12575)


In [5]:
df_consolidado.drop(columns='Unnamed: 0', inplace=True)

In [6]:
# Calcula a Similaridade de Cosseno
# linear_kernel é um método mais rápido para calcular a similaridade de cosseno em matrizes esparsas (como o TF-IDF).
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

print("\nMatriz de Similaridade de Cosseno calculada. Formato (Filme x Filme):")
print(cosine_sim.shape)


Matriz de Similaridade de Cosseno calculada. Formato (Filme x Filme):
(31030, 31030)


In [7]:
# Cria uma Série mapeando o Título Original ao Índice do DataFrame
indices = pd.Series(df_consolidado.index, index=df_consolidado['Título Original']).drop_duplicates()

def get_content_recommendations(title, cosine_sim=cosine_sim, df=df_consolidado, indices=indices):
    
    # 1. Obter o índice do filme que corresponde ao título
    idx = indices[title]

    # 2. Obter as pontuações de similaridade de todos os filmes com aquele filme
    sim_scores = list(enumerate(cosine_sim[idx]))

    # 3. Ordenar os filmes com base nas pontuações de similaridade
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # 4. Obter os 10 filmes mais similares (o primeiro é o próprio filme)
    sim_scores = sim_scores[1:11]

    # 5. Obter os índices dos filmes e os títulos
    movie_indices = [i[0] for i in sim_scores]
    
    # 6. Retornar os títulos dos 10 filmes mais similares
    return df['Título Original'].iloc[movie_indices]

# --- TESTE ---
# Tente recomendar filmes similares a 'Toy Story'
print("\n--- Filmes Similares a 'Toy Story' (Baseado no Conteúdo) ---")
print(get_content_recommendations('Toy Story'))


--- Filmes Similares a 'Toy Story' (Baseado no Conteúdo) ---
19910               Toy Story That Time Forgot
22391          Barbie and the Three Musketeers
1572                            Small Soldiers
12855                              Toy Story 3
6526                                     Dolls
1642                              Child's Play
1871                                      Toys
28544                                   Trolls
3389               The Transformers: The Movie
12671    LEGO: The Adventures of Clutch Powers
Name: Título Original, dtype: object


In [14]:
print(get_content_recommendations('Ant-Man'))

20405                           Captain America: Civil War
20396                              Avengers: Age of Ultron
12705                                           Iron Man 2
29207                        Marvel One-Shot: Agent Carter
21608                             Marvel One-Shot: Item 47
14634                                         The Avengers
18078                  Captain America: The Winter Soldier
20406                                       Doctor Strange
17394                                 Thor: The Dark World
29211    Marvel One-Shot: A Funny Thing Happened on the...
Name: Título Original, dtype: object


In [16]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import linear_kernel

# indices = pd.Series(df_consolidado.index, index=df_consolidado['Título Original']).drop_duplicates()
# tfidf_matrix (A matriz de todos os filmes, já calculada)

def get_historical_recommendations(liked_movies, tfidf_matrix=tfidf_matrix, df=df_consolidado, indices=indices, top_n=10):
    
    # 1. Encontrar os índices dos filmes que o usuário gostou
    movie_indices = [indices[title] for title in liked_movies if title in indices]
    
    if not movie_indices:
        return "Nenhum filme encontrado na base de dados."

    # 2. Calcular o Vetor de Gosto Médio do Usuário

    # Pega os vetores da tfidf_matrix e os converte explicitamente para um array NumPy
    # O .toarray() converte a matriz esparsa (sparse matrix) em um array denso (numpy array)
    # O .mean(axis=0) calcula a média dos vetores dos filmes gostados
    user_profile_vector = np.asarray(tfidf_matrix[movie_indices].toarray()).mean(axis=0)

    # 3. Calcular a Similaridade de Cosseno (Usuário vs. Todos os Filmes)
    # O vetor do usuário precisa ser remodelado (reshape) para (1, N) para a comparação
    user_profile_vector = user_profile_vector.reshape(1, -1)
    
    # Compara o vetor do usuário (1 linha) com a matriz de todos os filmes (N linhas)
    cosine_sim = linear_kernel(user_profile_vector, tfidf_matrix)

    # 4. Cria a lista de scores e ordena
    sim_scores = list(enumerate(cosine_sim[0]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # 5. Retorna o Top N (excluindo os filmes que já estão na lista de gostos, se possível)
    
    # Filtra os filmes que já foram usados no cálculo
    final_recommendations = []
    for index, score in sim_scores:
        title = df.loc[index, 'Título Original']
        if title not in liked_movies:
            final_recommendations.append(title)
        if len(final_recommendations) >= top_n:
            break
            
    return final_recommendations

# --- TESTE ---
# Simulando que o usuário gostou dos filmes 'Toy Story' e 'Small Soldiers'
filmes_gostados = ['Toy Story', 'Small Soldiers']
print(f"\n--- Recomendações Baseadas em Histórico ({filmes_gostados}) ---")
print(get_historical_recommendations(filmes_gostados))


--- Recomendações Baseadas em Histórico (['Toy Story', 'Small Soldiers']) ---
['Toy Story That Time Forgot', 'Barbie and the Three Musketeers', 'Toys', 'Toy Story 3', 'Dolls', "Child's Play 2", "Child's Play", 'The Transformers: The Movie', 'LEGO: The Adventures of Clutch Powers', 'Alma']


In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import linear_kernel

# Funções e DataFrames necessários:
# - tfidf (o objeto TfidfVectorizer já treinado)
# - tfidf_matrix (Matriz de conteúdo dos filmes)
# - df_consolidado (DataFrame com 'Título Original', 'Data de Lançamento', etc.)
# - indices (Série mapeando Título Original -> Índice)


def get_final_recommendations(liked_movies, current_preference_text, user_age, 
                              tfidf_matrix=tfidf_matrix, df=df_consolidado, indices=indices):
    
    # --- 1. VETORIZAÇÃO DO HISTÓRICO (VETOR 1) ---
    movie_indices = [indices[title] for title in liked_movies if title in indices]
    
    if not movie_indices:
        # Se não houver histórico, o vetor inicial é zero, mas é somado à preferência atual.
        history_vector = np.zeros(tfidf_matrix.shape[1])
    else:
        # Vetor de gosto médio (Histórico) - Converte para array antes de calcular a média
        history_vector = np.asarray(tfidf_matrix[movie_indices].toarray()).mean(axis=0)

    # --- 2. VETORIZAÇÃO DA PREFERÊNCIA ATUAL (VETOR 2) ---
    
    # Vetor TF-IDF dos termos que o usuário digitou (ex: "ação comédia")
    preference_vector = tfidf.transform([current_preference_text]).toarray()[0]
    
    
    # --- 3. FUSÃO DE GOSTO: CRIAÇÃO DO PERFIL FINAL ---
    
    # Soma os vetores de histórico e preferência atual.
    # A simples soma é uma forma comum de dar peso igual aos dois fatores.
    final_user_vector = history_vector + preference_vector
    
    # Normaliza e formata o vetor para o cálculo de similaridade (1 linha, N colunas)
    final_user_vector = final_user_vector.reshape(1, -1)
    
    
    # --- 4. CÁLCULO DA SIMILARIDADE E PONDERAÇÃO ---

    # 4.1. Similaridade de Cosseno (Perfil Final vs. Todos os Filmes)
    user_cosine_sim = linear_kernel(final_user_vector, tfidf_matrix)
    sim_scores = list(enumerate(user_cosine_sim[0]))
    
    # 4.2. Ponderação por RECÊNCIA/IDADE
    # Usuário de 21 anos provavelmente prefere filmes mais recentes (Idade do Filme ~ 1 ano)
    
    # Calcula a idade do filme em dias e normaliza (quanto menor a idade do filme, maior o 'recency_bonus')
    MAX_IDADE = (pd.to_datetime('today') - df_consolidado['Data de Lançamento']).dt.days.max()
    df_consolidado['recency_bonus'] = 1 - ( (pd.to_datetime('today') - pd.to_datetime(df_consolidado['Data de Lançamento'])).dt.days / MAX_IDADE )
    
    # Criamos um fator de preferência alta para a recência (ex: 0.1)
    RECENCY_WEIGHT = 0.1 
    
    final_scores = []
    for index, score in sim_scores:
        # O bônus é maior para filmes mais recentes
        bonus = df_consolidado.loc[index, 'recency_bonus'] * RECENCY_WEIGHT 
        final_score = score + bonus
        final_scores.append((index, final_score))
        
    # --- 5. RANKING E FILTRAGEM ---

    # Ordena pelo novo score final (similaridade + bônus de recência)
    final_scores = sorted(final_scores, key=lambda x: x[1], reverse=True)

    # Obter os 5 filmes mais bem ranqueados (excluindo filmes do histórico)
    top_5_indices = []
    for index, score in final_scores:
        if df.loc[index, 'Título Original'] not in liked_movies:
            top_5_indices.append(index)
        if len(top_5_indices) >= 5:
            break
            
    # --- 6. RETORNO ---
    return df.loc[top_5_indices, ['Título Original', 'genres_list', 'Data de Lançamento']]


--- RECOMENDAÇÃO FINAL PERSONALIZADA ---
                       Título Original                     genres_list  \
19910       Toy Story That Time Forgot                Animation|Family   
22391  Barbie and the Three Musketeers                Animation|Family   
12855                      Toy Story 3         Animation|Family|Comedy   
1871                              Toys  Fantasy|Comedy|Science Fiction   
6526                             Dolls                  Fantasy|Horror   

      Data de Lançamento  
19910         2014-12-02  
22391         2009-09-15  
12855         2010-06-16  
1871          1992-12-18  
6526          1987-03-01  


In [24]:

# --- TESTE FINAL (Exemplo de Uso) ---

# Simulando uma consulta:
# Gosto de 'Toy Story' e 'Small Soldiers' (Histórico)
# Quero ver algo com o termo 'aventura' (Preferência Atual)
# Minha idade é 21 (afeta o bônus de Recência)

filmes_gostados = ['Spider-Man', 'Ant-Man', 'Back to the Future']
preferencia_texto = "Comedy Science Fiction Science" 
idade_usuario = 21

print("--- RECOMENDAÇÃO FINAL PERSONALIZADA ---")
df_consolidado['Data de Lançamento'] = pd.to_datetime(df_consolidado['Data de Lançamento'], errors='coerce')

recomendacoes = get_final_recommendations(
    liked_movies=filmes_gostados,
    current_preference_text=preferencia_texto,
    user_age=idade_usuario
)
display(recomendacoes)

--- RECOMENDAÇÃO FINAL PERSONALIZADA ---


,Título Original,genres_list,Data de Lançamento
5482,The Spirit of '76,Comedy|Science Fiction,1990-01-01
27691,One Day Like Rain,Drama|Science Fiction,2007-06-15
13620,ナイスの森 The First Contact,Comedy|Drama|Science Fiction,2005-10-25
11093,"20,000 Leagues Under the Sea",Adventure|Drama|Action|Science Fiction,1916-12-24
30717,Órbita 9,Drama|Science Fiction|Romance,2017-04-07
